## test random environment with openai gym

In [2]:
import gym
import random

In [3]:
env = gym.make('CartPole-v1')
states = env.observation_space.shape[0]
actions = env.action_space.n

In [4]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = random.choice([0,1])
        n_state, reward, done, info = env.step(action)
        score += reward
    print(f'Episode: {episode} Score: {score}')

Episode: 1 Score: 33.0
Episode: 2 Score: 35.0
Episode: 3 Score: 13.0
Episode: 4 Score: 15.0
Episode: 5 Score: 20.0
Episode: 6 Score: 19.0
Episode: 7 Score: 8.0
Episode: 8 Score: 28.0
Episode: 9 Score: 30.0
Episode: 10 Score: 11.0


In [5]:
env.close()

## create a deep learning model with keras

In [12]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [13]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [14]:
model = build_model(states, actions)

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 4)                 0         
                                                                 
 dense_3 (Dense)             (None, 24)                120       
                                                                 
 dense_4 (Dense)             (None, 24)                600       
                                                                 
 dense_5 (Dense)             (None, 2)                 50        
                                                                 
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


## build agent with keras-rl

In [16]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [17]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [18]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 13:57 - reward: 1.0000

C:\Users\amoel.geogy\AppData\Local\Continuum\anaconda3\envs\reinfenv\lib\site-packages\keras\engine\training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
C:\Users\amoel.geogy\AppData\Local\Continuum\anaconda3\envs\reinfenv\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 62s 6ms/step - reward: 1.0000
88 episodes - episode_reward: 112.432 [10.000, 460.000] - loss: 2.322 - mae: 19.773 - mean_q: 40.133

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 70s 7ms/step - reward: 1.0000
45 episodes - episode_reward: 220.022 [167.000, 377.000] - loss: 3.791 - mae: 40.868 - mean_q: 82.766

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 70s 7ms/step - reward: 1.0000
35 episodes - episode_reward: 291.571 [184.000, 500.000] - loss: 3.154 - mae: 46.003 - mean_q: 92.880

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 76s 8ms/step - reward: 1.0000
31 episodes - episode_reward: 313.452 [230.000, 415.000] - loss: 2.042 - mae: 47.266 - mean_q: 95.261

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 71s 7ms/step - reward: 1.0000
done, took 348.786 seconds


In [19]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 500.000, steps: 500
Episode 2: reward: 500.000, steps: 500
Episode 3: reward: 500.000, steps: 500
Episode 4: reward: 500.000, steps: 500
Episode 5: reward: 500.000, steps: 500
Episode 6: reward: 500.000, steps: 500
Episode 7: reward: 500.000, steps: 500
Episode 8: reward: 500.000, steps: 500
Episode 9: reward: 500.000, steps: 500
Episode 10: reward: 500.000, steps: 500
Episode 11: reward: 500.000, steps: 500
Episode 12: reward: 476.000, steps: 476
Episode 13: reward: 500.000, steps: 500
Episode 14: reward: 500.000, steps: 500
Episode 15: reward: 500.000, steps: 500
Episode 16: reward: 500.000, steps: 500
Episode 17: reward: 500.000, steps: 500
Episode 18: reward: 500.000, steps: 500
Episode 19: reward: 500.000, steps: 500
Episode 20: reward: 500.000, steps: 500
Episode 21: reward: 500.000, steps: 500
Episode 22: reward: 500.000, steps: 500
Episode 23: reward: 500.000, steps: 500
Episode 24: reward: 500.000, steps: 500
Episode 25: reward: 

In [20]:
_ = dqn.test(env, nb_episodes=1, visualize=True, )

Testing for 1 episodes ...
Episode 1: reward: 500.000, steps: 500


In [21]:
env.close()

In [22]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [23]:
del model
del dqn
del env

## reloading agent from memory

In [24]:
# dqn is deleted
dqn.test(env, nb_episodes=5, visualize=True)

NameError: name 'dqn' is not defined

In [25]:
# reinstantiate the model
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [26]:
# reload the saved weights
dqn.load_weights('dqn_weights.h5f')

In [33]:
_ = dqn.test(env, nb_episodes=1, visualize=True)

Testing for 1 episodes ...
Episode 1: reward: 200.000, steps: 200


In [28]:
env.close()